In [1]:
import pandas as pd
import cvxpy

from linestar import ProjectionData

In [2]:
proj = ProjectionData()

/home/eadains/DFS/linestar.py:54: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 54 of the file /home/eadains/DFS/linestar.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


In [3]:
slate = proj.slate
positions = proj.pos_mat
teams = proj.teams
games = proj.games

In [25]:
slate.loc[slate["Position"] == "P"]

,Player,Position,Team,Opponent,Order,Salary,Projection,Consensus,pOwn,Game,Opp_Pitcher
0,Aaron Nola,P,PHI,@ATL,NaN,8900,31.85,37.18,0.46,PHI@ATL,NaN
1,Shohei Ohtani,P,LAA,vs TOR,NaN,9900,33.56,36.09,0.51,TOR@LAA,NaN
2,Frankie Montas,P,OAK,vs TEX,NaN,8800,39.78,34.26,0.63,TEX@OAK,NaN
3,Tarik Skubal,P,DET,vs CLE,NaN,10100,32.38,30.41,0.60,CLE@DET,NaN
4,Kyle Wright,P,ATL,vs PHI,NaN,9500,28.36,30.40,0.58,PHI@ATL,NaN
5,Nestor Cortes,P,NYY,@TB,NaN,10300,38.34,29.96,0.66,NYY@TB,NaN
6,Martín Pérez,P,TEX,@OAK,NaN,9700,32.58,29.54,0.62,TEX@OAK,NaN
7,Eric Lauer,P,MIL,@STL,NaN,10600,35.66,28.94,0.61,MIL@STL,NaN
8,Adam Wainwright,P,STL,vs MIL,NaN,9100,32.94,28.08,0.63,MIL@STL,NaN
9,Germán Márquez,P,COL,@WSH,NaN,7600,26.64,25.91,0.60,COL@WSH,NaN


In [21]:
selection = cvxpy.Variable(len(slate), boolean=True)
teams_var = cvxpy.Variable(len(teams.columns), boolean=True)
games_var = cvxpy.Variable(len(games.columns), boolean=True)

# Total salary must be less than or equal to $35,000
salary = selection @ slate["Salary"] <= 35000

# Must select players from at least 3 different teams
teams_var_con = teams_var <= selection @ teams
teams_con = cvxpy.sum(teams_var) >= 3

# Must select players from at least 2 different games
games_var_con = games_var <= selection @ games
games_con = cvxpy.sum(games_var) >= 2

# No more than 4 players, not counting the pitcher, can be selected from the same team
# First term is our selected players multiplied by a boolean array where 1's indicate non-pitcher players.
# This filters the selected players so the constraint only applies to non-pitcher players
players_teams = cvxpy.multiply(selection, ~positions["P"].astype(bool)) @ teams <= 4

# Must have 9 players selected
total_players = cvxpy.sum(selection) == 9

# Max and min number of players we can select for each position
# Must always have 1 pitcher, who cannot fill the UTIL position
# We can select up to 1 additional player from each other position because
# the second can fill the UTIL position
positions_max = [1, 2, 2, 2, 2, 4]
positions_min = [1, 1, 1, 1, 1, 3]
positions_max_con = selection @ positions <= positions_max
positions_min_con = selection @ positions >= positions_min

# Maxmize: Total number of fantasy points expcted for selected roster
tfp = selection @ slate["Consensus"]

In [22]:
constraints = [salary,
               teams_var_con,
               teams_con,
               games_var_con,
               games_con,
               players_teams,
               total_players,
               positions_max_con,
               positions_min_con]
problem = cvxpy.Problem(cvxpy.Maximize(tfp), constraints=constraints)
result = problem.solve(solver=cvxpy.GLPK_MI, verbose=True)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) May 26 04:20:44 PM: Your problem has 227 variables, 9 constraints, and 0 parameters.
(CVXPY) May 26 04:20:44 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 26 04:20:44 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 26 04:20:44 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 26 04:20:44 PM: Compiling problem (target solver=GLPK_MI).
(CVXPY) May 26 04:20:44 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> 

In [23]:
roster = dict(zip(slate["Player"], selection.value))
roster = [name for name, select in roster.items() if select > 0]

print(f"Expected Points: {selection.value @ slate['Projection']}\n")
print("Roster:")
for x in roster:
    print(x)

Expected Points: 122.05000000000001

Roster:
Aaron Nola
Byron Buxton
Mike Trout
Juan Soto
Freddie Freeman
AJ Pollock
Javier Báez
Chris Taylor
Jose Miranda
